# TF Dataset api

Try to use the new TF dataset api to setup the input pipeline

In [59]:
import tensorflow as tf
import numpy as np
import sys

from dataset import dataset_common
from preprocessing import ssd_preprocessing
from utility import anchor_manipulator
from utility import scaffolds

slim = tf.contrib.slim

In [60]:
VOC_LABELS = {
    'none': (0, 'Background'),
    'aeroplane': (1, 'Vehicle'),
    'bicycle': (2, 'Vehicle'),
    'bird': (3, 'Animal'),
    'boat': (4, 'Vehicle'),
    'bottle': (5, 'Indoor'),
    'bus': (6, 'Vehicle'),
    'car': (7, 'Vehicle'),
    'cat': (8, 'Animal'),
    'chair': (9, 'Indoor'),
    'cow': (10, 'Animal'),
    'diningtable': (11, 'Indoor'),
    'dog': (12, 'Animal'),
    'horse': (13, 'Animal'),
    'motorbike': (14, 'Vehicle'),
    'person': (15, 'Person'),
    'pottedplant': (16, 'Indoor'),
    'sheep': (17, 'Animal'),
    'sofa': (18, 'Indoor'),
    'train': (19, 'Vehicle'),
    'tvmonitor': (20, 'Indoor'),
}
# TODO: generate the training shards path automatically
training_files = ["/playground/data/VOCdevkit/tfdb/train-00000-of-00004", "/playground/data/VOCdevkit/tfdb/train-00001-of-00004",
                 "/playground/data/VOCdevkit/tfdb/train-00002-of-00004", "/playground/data/VOCdevkit/tfdb/train-00003-of-00004"]

In [76]:
def data_mapping_fn(example_proto, is_training, image_preprocessing_fn, anchor_encoder):
    keys_to_features = {
        'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.FixedLenFeature((), tf.string, default_value='jpeg'),
        'image/filename': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/height': tf.FixedLenFeature([1], tf.int64),
        'image/width': tf.FixedLenFeature([1], tf.int64),
        'image/channels': tf.FixedLenFeature([1], tf.int64),
        'image/shape': tf.FixedLenFeature([3], tf.int64),
        'image/object/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/label': tf.VarLenFeature(dtype=tf.int64),
        'image/object/bbox/difficult': tf.VarLenFeature(dtype=tf.int64),
        'image/object/bbox/truncated': tf.VarLenFeature(dtype=tf.int64),
    }
    items_to_handlers = {
        'image': slim.tfexample_decoder.Image('image/encoded', 'image/format'),
        'filename': slim.tfexample_decoder.Tensor('image/filename'),
        'shape': slim.tfexample_decoder.Tensor('image/shape'),
        'object/bbox': slim.tfexample_decoder.BoundingBox(
                ['ymin', 'xmin', 'ymax', 'xmax'], 'image/object/bbox/'),
        'object/label': slim.tfexample_decoder.Tensor('image/object/bbox/label'),
        'object/difficult': slim.tfexample_decoder.Tensor('image/object/bbox/difficult'),
        'object/truncated': slim.tfexample_decoder.Tensor('image/object/bbox/truncated'),
    }
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
    [org_image, filename, shape, glabels_raw, gbboxes_raw, isdifficult] = decoder.decode(example_proto, ['image', 'filename', 'shape',
                                                                     'object/label',
                                                                     'object/bbox',
                                                                     'object/difficult'])
    
    if is_training:
        # if all is difficult, then keep the first one
        isdifficult_mask =tf.cond(tf.count_nonzero(isdifficult, dtype=tf.int32) < tf.shape(isdifficult)[0],
                                lambda : isdifficult < tf.ones_like(isdifficult),
                                lambda : tf.one_hot(0, tf.shape(isdifficult)[0], on_value=True, off_value=False, dtype=tf.bool))

        glabels_raw = tf.boolean_mask(glabels_raw, isdifficult_mask)
        gbboxes_raw = tf.boolean_mask(gbboxes_raw, isdifficult_mask)

    # Pre-processing image, labels and bboxes.

    if is_training:
        image, glabels, gbboxes = image_preprocessing_fn(org_image, glabels_raw, gbboxes_raw)
    else:
        image = image_preprocessing_fn(org_image, glabels_raw, gbboxes_raw)
        glabels, gbboxes = glabels_raw, gbboxes_raw

    gt_targets, gt_labels, gt_scores = anchor_encoder(glabels, gbboxes)
    
    # return [image, filename, shape, gt_targets, gt_labels, gt_scores]
    return image, {'shape': shape, 'loc_targets': gt_targets, 'cls_targets': gt_labels, 'match_scores': gt_scores}

In [78]:
def input_pipeline(training_files='train-*', is_training=True, batch_size=FLAGS.batch_size):
    def input_fn():
        out_shape = [FLAGS.train_image_size] * 2
        anchor_creator = anchor_manipulator.AnchorCreator(out_shape,
                                                    layers_shapes = [(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)],
                                                    anchor_scales = [(0.1,), (0.2,), (0.375,), (0.55,), (0.725,), (0.9,)],
                                                    extra_anchor_scales = [(0.1414,), (0.2739,), (0.4541,), (0.6315,), (0.8078,), (0.9836,)],
                                                    anchor_ratios = [(1., 2., .5), (1., 2., 3., .5, 0.3333), (1., 2., 3., .5, 0.3333), (1., 2., 3., .5, 0.3333), (1., 2., .5), (1., 2., .5)],
                                                    layer_steps = [8, 16, 32, 64, 100, 300])
        all_anchors, all_num_anchors_depth, all_num_anchors_spatial = anchor_creator.get_all_anchors()

        num_anchors_per_layer = []
        for ind in range(len(all_anchors)):
            num_anchors_per_layer.append(all_num_anchors_depth[ind] * all_num_anchors_spatial[ind])

        anchor_encoder_decoder = anchor_manipulator.AnchorEncoder(allowed_borders = [1.0] * 6,
                                                            positive_threshold = FLAGS.match_threshold,
                                                            ignore_threshold = FLAGS.neg_threshold,
                                                            prior_scaling=[0.1, 0.1, 0.2, 0.2])

        image_preprocessing_fn = lambda image_, labels_, bboxes_ : ssd_preprocessing.preprocess_image(image_, labels_, bboxes_, out_shape, is_training=is_training, data_format=FLAGS.data_format, output_rgb=False)
        anchor_encoder_fn = lambda glabels_, gbboxes_: anchor_encoder_decoder.encode_all_anchors(glabels_, gbboxes_, all_anchors, all_num_anchors_depth, all_num_anchors_spatial)

        filenames = tf.placeholder(tf.string, shape=[None])
        arga = tf.constant(True)

        dataset = tf.data.TFRecordDataset(training_files)
        dataset = dataset.map(lambda x: data_mapping_fn(x, is_training, image_preprocessing_fn, anchor_encoder))
        dataset = dataset.repeat() # repeat the input infinitely
        dataset = dataset.batch(batch_size) # set the batch size
        iterator = dataset.make_initializable_iterator()
        
#         image, _, shape, loc_targets, cls_targets, match_scores = dataset_common.slim_get_batch(FLAGS.num_classes,
#                                                                                 batch_size,
#                                                                                 ('train' if is_training else 'val'),
#                                                                                 os.path.join(FLAGS.data_dir, dataset_pattern),
#                                                                                 FLAGS.num_readers,
#                                                                                 FLAGS.num_preprocessing_threads,
#                                                                                 image_preprocessing_fn,
#                                                                                 anchor_encoder_fn,
#                                                                                 num_epochs=FLAGS.train_epochs,
#                                                                                 is_training=is_training)
        global global_anchor_info
        global_anchor_info = {'decode_fn': lambda pred : anchor_encoder_decoder.decode_all_anchors(pred, num_anchors_per_layer),
                            'num_anchors_per_layer': num_anchors_per_layer,
                            'all_num_anchors_depth': all_num_anchors_depth }

        # return image, {'shape': shape, 'loc_targets': loc_targets, 'cls_targets': cls_targets, 'match_scores': match_scores}
        return dataset
    return input_fn

NameError: name 'FLAGS' is not defined

In [72]:
filenames = tf.placeholder(tf.string, shape=[None])
is_training = tf.placeholder(tf.bool)
arga = tf.constant(True)

dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(lambda x: data_mapping_fn(x, is_training))
dataset = dataset.repeat() # repeat the input infinitely
dataset = dataset.batch(32) # set the batch size
iterator = dataset.make_initializable_iterator()

In [73]:
# init iterator with training data
sess = tf.Session()
sess.run(iterator.initializer, feed_dict={filenames: training_files, is_training: True})

# print iterator.output_classes
# print iterator.output_shapes
print sess.run(iterator.get_next())

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [500,307,3] and element 1 had shape [333,500,3].
	 [[Node: IteratorGetNext_6 = IteratorGetNext[output_shapes=[[?,?,?,3], [?], [?,3], [?,?], [?,?,4], [?,?]], output_types=[DT_UINT8, DT_STRING, DT_INT64, DT_INT64, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator_8)]]

Caused by op u'IteratorGetNext_6', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 486, in start
    